In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('recommendation').getOrCreate()

In [ ]:
from pyspark.ml.recommendation import ALS

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving movielens_ratings.csv to movielens_ratings.csv


In [ ]:
df = spark.read.csv('movielens_ratings.csv',inferSchema = True, header = True)

In [ ]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [ ]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



# **Develop an ALS model**

In [ ]:
training, test = df.randomSplit([0.8,0.2])

In [ ]:
als = ALS(maxIter = 5, regParam = 0.01, userCol = 'userId', itemCol = 'movieId', ratingCol = 'rating' )

In [ ]:
rec_model = als.fit(training)

# **Predict reviews**

In [ ]:
predictions = rec_model.transform(test)

In [ ]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    28| -2.3686156|
|      3|   1.0|    26|  0.5998629|
|      5|   2.0|    26|   2.122891|
|      6|   2.0|    22|  2.2316723|
|      0|   1.0|     6|-0.12526453|
|      1|   1.0|     3|  3.5235064|
|      1|   1.0|    20| 0.28392398|
|      4|   1.0|     5|   2.884355|
|      5|   1.0|     5|  0.9175768|
|      0|   1.0|    19|  1.5033432|
|      5|   2.0|    15|  1.1367275|
|      2|   1.0|    17|  2.4595544|
|      6|   1.0|     9|  1.1393869|
|      0|   3.0|    10|  1.1568745|
|      2|   4.0|    10|   2.526958|
|      4|   1.0|    24|  1.3968246|
|      5|   1.0|    29|  2.0766542|
|      0|   1.0|    21|  1.5706073|
|      2|   4.0|    21|  1.6127526|
|      1|   1.0|    14|0.027819008|
+-------+------+------+-----------+
only showing top 20 rows



In [ ]:
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'rating', predictionCol = 'prediction')

In [ ]:
rmse = evaluator.evaluate(predictions)

In [ ]:
print('RMSE')
print(rmse)

RMSE
1.8223004394562319


# **Look for review predictions of a particular user**

In [ ]:
single_user = test.filter(test['userId'] == 11).select(['movieId','userId'])

In [ ]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     13|    11|
|     23|    11|
|     25|    11|
|     37|    11|
|     39|    11|
|     48|    11|
|     59|    11|
|     70|    11|
|     76|    11|
|     81|    11|
|     88|    11|
+-------+------+



In [ ]:
recommendations = rec_model.transform(single_user)

In [ ]:
recommendations.orderBy('prediction', ascending = False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     25|    11|  4.7116127|
|     23|    11|  3.2164128|
|     48|    11|  2.9930258|
|     88|    11|  2.7399864|
|     13|    11|  2.5537074|
|     39|    11|   2.551756|
|     81|    11|  2.4103136|
|     37|    11|  2.3984385|
|     59|    11|   2.346611|
|     76|    11|  1.8911073|
|     70|    11|-0.12439114|
+-------+------+-----------+

